In [93]:
import pandas as pd
import yfinance as yf
import datetime
import calendar

## Fetch AAPL stock from yfinance

In [187]:
start_date = "2010-01-01"
end_date = "2024-12-31"
symbol = "AAPL"
stock_data = yf.download(symbol, start = start_date, end = end_date)
stock_data.reset_index(inplace = True)
stock_data.columns = ["date", "close", "high", "low", "open", "volume"]

[*********************100%***********************]  1 of 1 completed


In [188]:
stock_data.head()

,date,close,high,low,open,volume
0,2010-01-04,6.440331,6.455077,6.391279,6.422877,493729600
1,2010-01-05,6.451466,6.487879,6.417459,6.458086,601904800
2,2010-01-06,6.348847,6.477046,6.342226,6.451466,552160000
3,2010-01-07,6.337110,6.379844,6.291067,6.372320,477131200
4,2010-01-08,6.379241,6.379844,6.291369,6.328684,447610800


## Derive month expiry dates: They are usually the third friday of the month

In [189]:
#Iterate through each year from 2011 to 2024 and each month
#derive the first day of the month and then find after how many days the third friday comes
#This would be days to first friday from start of the month + 14 days(for next two fridays)
#eg. Let's say first day of the month is Sunday, next friday would be in 5 days. we add two more weeks ie 14 days.
#We get 5+14 = 19 which is the third friday of the month

third_fridays_of_month_dict = {"date": []}

for i in range(2010, 2025):
    for j in range(1,13):
        first_day_date = datetime.date(i, j, 1)
        first_day_weekday = first_day_date.weekday()
        days_to_friday_from_first_day = (calendar.FRIDAY - first_day_weekday)%7
        third_friday_of_month_date = first_day_date + datetime.timedelta(days=(days_to_friday_from_first_day + 14))
        third_fridays_of_month_dict["date"].append(third_friday_of_month_date)

third_fridays_of_month = pd.DataFrame(third_fridays_of_month_dict)
third_fridays_of_month["date"] = pd.to_datetime(third_fridays_of_month["date"])
third_fridays_of_month.shape
        

(180, 1)

In [190]:
third_fridays_of_month.dtypes

date    datetime64[ns]
dtype: object

In [191]:
#let's find out holidays on the expiry date. If the the third friday of the month doesn't exist in stock_data, then it was holiday.
#We have to go back one day for those dates
merged = third_fridays_of_month.merge(stock_data, on = "date", how = "left", indicator = True)
print(merged[merged["_merge"] == "left_only"])

          date  close  high  low  open  volume     _merge
51  2014-04-18    NaN   NaN  NaN   NaN     NaN  left_only
111 2019-04-19    NaN   NaN  NaN   NaN     NaN  left_only
147 2022-04-15    NaN   NaN  NaN   NaN     NaN  left_only


In [192]:
#So we have 3 days when expiry had a holiday, we reduce 1 day from those dates
holiday_expiry_dates = merged.loc[merged['_merge'] == 'left_only'].copy()
mask = third_fridays_of_month["date"].isin(holiday_expiry_dates["date"])
third_fridays_of_month.loc[mask, "date"] = third_fridays_of_month.loc[mask, "date"] + pd.Timedelta(days = -1)

#Check again
merged = third_fridays_of_month.merge(stock_data, on = "date", how = "left", indicator = True)
print(merged[merged["_merge"] == "left_only"])

Empty DataFrame
Columns: [date, close, high, low, open, volume, _merge]
Index: []


In [193]:
#Add a column to stock_data to identify the expiry day
stock_data['expiry'] = ""
stock_data.loc[stock_data["date"].isin(third_fridays_of_month["date"]), "expiry"] = "expiry_day"
stock_data[stock_data["expiry"] == "expiry_day"].head()

,date,close,high,low,open,volume,expiry
9,2010-01-15,6.197176,6.367807,6.195370,6.347644,594067600,expiry_day
32,2010-02-19,6.068976,6.115020,6.052124,6.074695,415469600,expiry_day
52,2010-03-19,6.688303,6.778283,6.657608,6.764741,559445600,expiry_day
71,2010-04-16,7.445155,7.557705,7.359388,7.480364,750545600,expiry_day
96,2010-05-21,7.292284,7.357887,6.962156,7.006393,1223891200,expiry_day


In [194]:
##Adding first day of the new expiry contract, it will start after the previous expiry date
stock_data.sort_values(by = "date").reset_index(drop = True)
for i in stock_data.index:
    if stock_data.at[i, "expiry"] == "expiry_day":
        stock_data.at[i+1, "expiry"] = "new_day"
stock_data[(stock_data["expiry"] == "expiry_day") | (stock_data["expiry"] == "new_day")].head()

,date,close,high,low,open,volume,expiry
9,2010-01-15,6.197176,6.367807,6.195370,6.347644,594067600,expiry_day
10,2010-01-19,6.471327,6.475841,6.236597,6.269399,730007600,new_day
32,2010-02-19,6.068976,6.115020,6.052124,6.074695,415469600,expiry_day
33,2010-02-22,6.031359,6.093954,5.994344,6.089139,390563600,new_day
52,2010-03-19,6.688303,6.778283,6.657608,6.764741,559445600,expiry_day


## Adding columns for option contract

We first find the strike price interval
At a new day, find a strike price above 5% and above 10% for new_day, that's where we would be taking our trades

In [195]:
stock_data.dtypes

date      datetime64[ns]
close            float64
high             float64
low              float64
open             float64
volume             int64
expiry            object
dtype: object

In [196]:
#Add a new column to find strike price intervals
stock_data = stock_data.round(2)
stock_data["strike_price_intervals"] = ""

for i in stock_data[stock_data["expiry"] == "new_day"].index:
    if stock_data.loc[i, "close"] <= 25.00:
        stock_data.loc[i, "strike_price_intervals"] = 0.50
    elif stock_data.loc[i, "close"] > 25.00 and stock_data.loc[i, "close"] <= 200.00:
        stock_data.loc[i, "strike_price_intervals"] = 1.00
    elif stock_data.loc[i, "close"] > 200.00:
        stock_data.loc[i, "strike_price_intervals"] = 2.5

In [197]:
stock_data[stock_data["expiry"] == "new_day"].head()

,date,close,high,low,open,volume,expiry,strike_price_intervals
10,2010-01-19,6.47,6.48,6.24,6.27,730007600,new_day,0.5
33,2010-02-22,6.03,6.09,5.99,6.09,390563600,new_day,0.5
53,2010-03-22,6.76,6.80,6.63,6.63,456419600,new_day,0.5
72,2010-04-19,7.44,7.46,7.28,7.43,566924400,new_day,0.5
97,2010-05-24,7.43,7.55,7.41,7.44,754238800,new_day,0.5


In [198]:
##Choose strike prices at 5% and 10%
stock_data["sp_5per_away"]  = ""
stock_data["sp_10per_away"] = ""

sp_for_expiry_5per = 0
sp_for_expiry_10per = 0
for i in stock_data[(stock_data["expiry"] == "new_day") | (stock_data["expiry"] == "expiry_day")].index:
    if stock_data.loc[i, "expiry"] == "new_day":
        interval = stock_data.loc[i, "strike_price_intervals"]
        five_per_away = stock_data.loc[i, "open"] * 1.05
        ten_per_away = stock_data.loc[i, "open"] * 1.10
        stock_data.loc[i, "sp_5per_away"] = ((five_per_away//interval) + 1) * interval
        stock_data.loc[i, "sp_10per_away"] = ((ten_per_away//interval) + 1) * interval
        sp_for_expiry_5per = stock_data.loc[i, "sp_5per_away"]
        sp_for_expiry_10per = stock_data.loc[i, "sp_10per_away"]
    else:
        stock_data.loc[i, "sp_5per_away"] = sp_for_expiry_5per
        stock_data.loc[i, "sp_10per_away"] = sp_for_expiry_10per

stock_data[(stock_data["expiry"] == "new_day") | (stock_data["expiry"] == "expiry_day")]

,date,close,high,low,open,volume,expiry,strike_price_intervals,sp_5per_away,sp_10per_away
9,2010-01-15,6.20,6.37,6.20,6.35,594067600,expiry_day,,0,0
10,2010-01-19,6.47,6.48,6.24,6.27,730007600,new_day,0.5,7.0,7.0
32,2010-02-19,6.07,6.12,6.05,6.07,415469600,expiry_day,,7.0,7.0
33,2010-02-22,6.03,6.09,5.99,6.09,390563600,new_day,0.5,6.5,7.0
52,2010-03-19,6.69,6.78,6.66,6.76,559445600,expiry_day,,6.5,7.0
...,...,...,...,...,...,...,...,...,...,...
3724,2024-10-21,235.96,236.33,233.94,233.94,36254500,new_day,2.5,247.5,257.5
3743,2024-11-15,224.75,226.67,224.02,226.15,47923700,expiry_day,,247.5,257.5
3744,2024-11-18,227.77,229.49,224.92,225.00,44686000,new_day,2.5,237.5,250.0
3767,2024-12-20,254.21,254.72,245.42,247.77,147495300,expiry_day,,237.5,250.0


## Calculating the approximate option price

We don't have historic data for option price at particular strike price so we will approximate the option price at new day.<br>
First we find the annualized volatility for each day<br>
Then we use formula Adjusted Premium=S * σ * T⋅* Discount<br>
S = price of the stock<br>
σ = Annualized volatility<br>
T = sqrt(time to expiry / 365)<br>
Discount = We multiply by a factor to depending on the strike price, the further it is from current value, more is the discount<br>

In [199]:
##Find annualized volatility for each day
import numpy as np

stock_data["daily_returns"] = np.log(stock_data["close"]/stock_data["close"].shift(1))
stock_data["annualized_vol"] = daily_returns.rolling(window = 252).std() * np.sqrt(252)
stock_data[stock_data["date"] >= "2012-01-01"]

,date,close,high,low,open,volume,expiry,strike_price_intervals,sp_5per_away,sp_10per_away,daily_returns,annualized_vol
504,2012-01-03,12.38,12.41,12.31,12.32,302220800,,,,,0.015466,0.261626
505,2012-01-04,12.44,12.48,12.32,12.34,260022000,,,,,0.004835,0.261593
506,2012-01-05,12.58,12.60,12.42,12.49,271269600,,,,,0.011191,0.261479
507,2012-01-06,12.71,12.72,12.62,12.63,318292800,,,,,0.010281,0.261235
508,2012-01-09,12.69,12.87,12.68,12.80,394024400,,,,,-0.001575,0.259841
...,...,...,...,...,...,...,...,...,...,...,...,...
3768,2024-12-23,254.99,255.37,253.17,254.49,40858800,new_day,2.5,267.5,280.0,0.003064,NaN
3769,2024-12-24,257.92,257.93,255.01,255.21,23234700,,,,,0.011425,NaN
3770,2024-12-26,258.74,259.81,257.35,257.91,27237100,,,,,0.003174,NaN
3771,2024-12-27,255.31,258.42,252.78,257.55,42355300,,,,,-0.013345,NaN


In [200]:
#We only keep the data starting from the first new day of 2012 and ending at the last expiry day of 2024
stock_data = stock_data[stock_data["date"] >= "2012-01-01"]

first_new_day_index = stock_data[stock_data["expiry"] == "new_day"].index.min()
last_expiry_day = stock_data[stock_data["expiry"] == "expiry_day"].index.max()

print(first_new_day_index, last_expiry_day)
stock_data = stock_data.loc[first_new_day_index:last_expiry_day].reset_index(drop = True)

stock_data.head()

517 3767


,date,close,high,low,open,volume,expiry,strike_price_intervals,sp_5per_away,sp_10per_away,daily_returns,annualized_vol
0,2012-01-23,12.86,12.89,12.71,12.72,306062400,new_day,0.5,13.5,14.0,0.016465,0.265360
1,2012-01-24,12.65,12.79,12.63,12.79,547638000,,,,,-0.016465,0.265158
2,2012-01-25,13.44,13.68,13.35,13.68,958314000,,,,,0.060578,0.265496
3,2012-01-26,13.38,13.51,13.34,13.49,323985200,,,,,-0.004474,0.267372
4,2012-01-27,13.46,13.50,13.35,13.37,299709200,,,,,0.005961,0.267122


In [201]:
## we only keep the new_day and expiry_day rows
## calculate days to expiry from new day to next expiry, this will be used to approximate option price

stock_data = stock_data[(stock_data["expiry"] == "expiry_day") | (stock_data["expiry"] == "new_day")]
stock_data.sort_values(by='date')
stock_data = stock_data.reset_index(drop = True)
stock_data["days_to_expiry"] = stock_data["date"].shift(-1) - stock_data["date"]
stock_data["days_to_expiry"] = stock_data["days_to_expiry"].dt.days
stock_data["days_to_expiry"] = stock_data["days_to_expiry"].astype("Int64")
stock_data.loc[stock_data["expiry"] == "expiry_day", "days_to_expiry"] = 0
stock_data

,date,close,high,low,open,volume,expiry,strike_price_intervals,sp_5per_away,sp_10per_away,daily_returns,annualized_vol,days_to_expiry
0,2012-01-23,12.86,12.89,12.71,12.72,306062400,new_day,0.5,13.5,14.0,0.016465,0.265360,25
1,2012-02-17,15.11,15.28,15.06,15.14,535805200,expiry_day,,13.5,14.0,0.000000,0.266360,0
2,2012-02-21,15.49,15.49,15.17,15.25,605595200,new_day,0.5,16.5,17.0,0.024838,0.266041,24
3,2012-03-16,17.62,17.73,17.39,17.60,825487600,expiry_day,,16.5,17.0,0.000000,0.264330,0
4,2012-03-19,18.09,18.11,17.73,18.01,901236000,new_day,0.5,19.0,20.0,0.026325,0.265611,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,2024-10-18,234.48,235.66,233.50,235.66,46431500,expiry_day,,240.0,250.0,0.012186,0.225342,0
306,2024-10-21,235.96,236.33,233.94,233.94,36254500,new_day,2.5,247.5,257.5,0.006292,0.224544,25
307,2024-11-15,224.75,226.67,224.02,226.15,47923700,expiry_day,,247.5,257.5,-0.014225,0.224815,0
308,2024-11-18,227.77,229.49,224.92,225.00,44686000,new_day,2.5,237.5,250.0,0.013348,0.225045,32


Let's calculate the option price for expiry = new_day<br>
Since we don't have all the data, we will use some assumptions<br><br>

We need to install library py_vollib<br><br>

Assumptions:<br>
Risk free interest rate would be 1%, ideally it should be actual historical date matching the options term<br>
Divided yeild would be 0.5% or 0.005<br>
Historical volatility = Implied volatility (Implied volatility is forward looking)<br>

In [202]:
pip install py_vollib

Note: you may need to restart the kernel to use updated packages.


In [203]:
#----preparing inputs for Black-Scholes-Merton (BSM) model
risk_free_rate = 0.01
dividend_yield = 0.005

# S: Underlying Price (Using 'open_price' as proxy)
S = stock_data['open']

# K: Strike Price
K_at_5per = stock_data['sp_5per_away']
K_at_10per = stock_data['sp_10per_away']

# T: Time to Expiration (in years)
# Using 365 days per year is standard for BSM time calculations
T = stock_data['days_to_expiry'] / 365.0

# sigma: Annualized Volatility (Using 'annualized_vol' HV as proxy)
sigma = stock_data['annualized_vol']

# r: Risk-Free Rate (defined above)
r = risk_free_rate

# q: Dividend Yield (defined above)
q = dividend_yield

In [204]:
from py_vollib.black_scholes_merton import black_scholes_merton
def calculate_premium_5per(row):
    #we provide the input, 'c' is for call option
    return black_scholes_merton('c', row['S'], row['K'], row['T'], r, row['sigma'], q)

In [205]:
#Create a temporary datafram just to calculate the premium price for both the strike price

temp_data_for_bsm_5per = pd.DataFrame({'S': S, 'K': K_at_5per, 'T': T, "sigma": sigma})
temp_data_for_bsm_10per = pd.DataFrame({'S': S, 'K': K_at_10per, 'T': T, "sigma": sigma})

In [206]:
temp_data_for_bsm_5per['premium_at_5per'] = temp_data_for_bsm_5per.apply(calculate_premium_5per, axis=1)
temp_data_for_bsm_10per['premium_at_10per'] = temp_data_for_bsm_10per.apply(calculate_premium_5per, axis=1)

In [207]:
temp_data_for_bsm_10per.head()

,S,K,T,sigma,premium_at_10per
0,12.72,14.0,0.068493,0.265360,0.035809
1,15.14,14.0,0.0,0.266360,1.140000
2,15.25,17.0,0.065753,0.266041,0.026261
3,17.60,17.0,0.0,0.264330,0.600000
4,18.01,20.0,0.087671,0.265611,0.064063


In [208]:
#merge premium prices to existing data

stock_data = pd.concat([stock_data, temp_data_for_bsm_5per['premium_at_5per']], axis = 1)
stock_data = pd.concat([stock_data, temp_data_for_bsm_10per['premium_at_10per']], axis = 1)

In [209]:
stock_data

,date,close,high,low,open,volume,expiry,strike_price_intervals,sp_5per_away,sp_10per_away,daily_returns,annualized_vol,days_to_expiry,premium_at_5per,premium_at_10per
0,2012-01-23,12.86,12.89,12.71,12.72,306062400,new_day,0.5,13.5,14.0,0.016465,0.265360,25,0.099629,0.035809
1,2012-02-17,15.11,15.28,15.06,15.14,535805200,expiry_day,,13.5,14.0,0.000000,0.266360,0,1.640000,1.140000
2,2012-02-21,15.49,15.49,15.17,15.25,605595200,new_day,0.5,16.5,17.0,0.024838,0.266041,24,0.067146,0.026261
3,2012-03-16,17.62,17.73,17.39,17.60,825487600,expiry_day,,16.5,17.0,0.000000,0.264330,0,1.100000,0.600000
4,2012-03-19,18.09,18.11,17.73,18.01,901236000,new_day,0.5,19.0,20.0,0.026325,0.265611,32,0.216627,0.064063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,2024-10-18,234.48,235.66,233.50,235.66,46431500,expiry_day,,240.0,250.0,0.012186,0.225342,0,0.000000,0.000000
306,2024-10-21,235.96,236.33,233.94,233.94,36254500,new_day,2.5,247.5,257.5,0.006292,0.224544,25,1.286411,0.314192
307,2024-11-15,224.75,226.67,224.02,226.15,47923700,expiry_day,,247.5,257.5,-0.014225,0.224815,0,0.000000,0.000000
308,2024-11-18,227.77,229.49,224.92,225.00,44686000,new_day,2.5,237.5,250.0,0.013348,0.225045,32,1.834131,0.389472


In [ ]:
#testing the modle

In [221]:
test_df =  yf.download(symbol, start = "2024-01-01", end = "2025-04-14")
test_df.reset_index(inplace = True)
test_df.columns = ["date", "close", "high", "low", "open", "volume"]

[*********************100%***********************]  1 of 1 completed


In [222]:
import numpy as np

test_df["daily_returns"] = np.log(test_df["close"]/test_df["close"].shift(1))
test_df["annualized_vol"] = test_df["daily_returns"].rolling(window = 252).std() * np.sqrt(252)
test_df[test_df["date"] >= "2012-01-01"]

,date,close,high,low,open,volume,daily_returns,annualized_vol
0,2024-01-02,184.532089,187.315382,182.792533,186.033072,82488700,NaN,NaN
1,2024-01-03,183.150375,184.770652,182.335262,183.120556,58414500,-0.007516,NaN
2,2024-01-04,180.824356,181.997307,179.800504,181.062914,71983600,-0.012781,NaN
3,2024-01-05,180.098694,181.669266,179.094727,180.903872,62303300,-0.004021,NaN
4,2024-01-08,184.452560,184.492330,180.416793,181.003268,59144500,0.023887,NaN
...,...,...,...,...,...,...,...,...
316,2025-04-07,181.460007,194.149994,174.619995,177.199997,160466300,-0.037426,0.277604
317,2025-04-08,172.419998,190.339996,169.210007,186.699997,120859500,-0.051102,0.282289
318,2025-04-09,198.850006,200.610001,171.889999,171.949997,184395900,0.142617,0.316147
319,2025-04-10,190.419998,194.779999,183.000000,189.070007,121880000,-0.043319,0.319123


In [227]:
test = black_scholes_merton('c', 198.149994, 210, 0.036, 0.0437, 0.321316, 0.005)

In [228]:
print(test)

1.180698868767489
